In [ ]:
#Task 1 : State Vector Simulation of Quantum Circuits

from typing import List, Dict
import numpy as np

class QuantumState:
    def __init__(self, amplitudes: List[complex]):
        self.amplitudes = np.array(amplitudes, dtype=complex)

class Ket(QuantumState):
    def __init__(self, amplitudes: List[complex]):
        super().__init__(amplitudes)
    
    def adjoint(self):
        return np.conjugate(self.amplitudes)

class Circuit:
    def __init__(self, size: int) -> None:
        self.size = size
        self.gates = []  

    def add_gate(self, gate: str, indices: List[int]):
        self.gates.append((gate, indices))

    def get_unitary_matrix(self) -> np.ndarray:
        unitary = np.identity(2 ** self.size)
        for gate, indices in self.gates:
            if gate == 'I':
                unitary = np.kron(unitary, np.identity(2))
            if gate == 'x':
                unitary = np.kron(unitary, np.array([[0, 1], [1, 0]]))
            if gate == 'h':
                unitary = np.kron(unitary, (1/np.sqrt(2)) * np.array([[1,1],[1,-1]]))
            elif gate == 'cx':
                unitary = np.kron(unitary, np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]]))
                
           
        return unitary[:2**self.size, :2**self.size]  


    
class ExecuteCircuit:
    def run_circuit(self, circuit: Circuit, state: Ket) -> np.ndarray:
        circ_mat = circuit.get_unitary_matrix()
        new_state = circ_mat @ state.amplitudes.reshape(-1, 1)  
        return new_state.flatten()  

# Example with initial two qubit state |00⟩
amplitudes = [1, 0, 0, 0]  # Initial state |0⟩
state = Ket(amplitudes)
my_circuit = Circuit(size=2)
my_circuit.add_gate('I', [0])
my_circuit.add_gate('x', [0])
my_circuit.add_gate('h', [0])
my_circuit.add_gate('cx', [0,1])
executor = ExecuteCircuit()
final_state = executor.run_circuit(my_circuit, state)

print("Final state after the circuit:", final_state)


In [ ]:
#Task 1 (b) : Advanced simulation using tensor multiplication 

from typing import List
import numpy as np
import time
import matplotlib.pyplot as plt

class QuantumState:
    def __init__(self, amplitudes: List[complex]):
        self.amplitudes = np.array(amplitudes, dtype=complex)

class Ket(QuantumState):
    def __init__(self, amplitudes: List[complex]):
        super().__init__(amplitudes)
    
    def adjoint(self):
        return np.conjugate(self.amplitudes)

class Circuit:
    def __init__(self, size: int) -> None:
        self.size = size
        self.gates = []

    def add_gate(self, gate: str, indices: List[int]):
        self.gates.append((gate, indices))

    def apply_gate(self, state_tensor, gate_matrix, qubit):
        axes = [qubit]
        reshaped_gate = gate_matrix.reshape([2] * 2)  
        
        return np.tensordot(state_tensor, reshaped_gate, axes=[axes, [0]])

    def run_circuit(self, state_tensor):
        for gate, indices in self.gates:
            if gate == 'I':
                gate_matrix = np.identity(2)
            elif gate == 'x':
                gate_matrix = np.array([[0, 1], [1, 0]], dtype=complex)
            elif gate == 'cx':
                gate_matrix = np.array([[1, 0, 0, 0],
                                        [0, 1, 0, 0],
                                        [0, 0, 0, 1],
                                        [0, 0, 1, 0]], dtype=complex)
            state_tensor = self.apply_gate(state_tensor, gate_matrix, indices[0])
        return state_tensor


def measure_runtime(max_qubits=10):
    runtimes = []
    qubit_counts = []

    for n_qubits in range(1, max_qubits + 1):
        initial_state = np.zeros([2] * n_qubits, dtype=complex)
        initial_state[(0,) * n_qubits] = 1.0  
        
       
        my_circuit = Circuit(size=n_qubits)
        for q in range(n_qubits):
            my_circuit.add_gate('x', [q])  

        start_time = time.time()
        final_state = my_circuit.run_circuit(initial_state)
        end_time = time.time()

    
        runtimes.append(end_time - start_time)
        qubit_counts.append(n_qubits)

    return qubit_counts, runtimes

# experiment 
qubit_counts, runtimes = measure_runtime()

plt.plot(qubit_counts, runtimes, marker='o')
plt.xlabel('Number of Qubits')
plt.ylabel('Runtime (seconds)')
plt.title('Runtime as a Function of Number of Qubits')
plt.show()
